In [3]:
import ppo
import numpy as np

def my_trained_model(observation):
    n_assets = observation.shape[1]
    return np.zeros(n_assets)

# Manually create `Args`
args = ppo.Args(
    env_id="PortfolioEnv-v0",
    total_timesteps=10000,
    num_envs=5,
    n_assets=3,
    window_size=10,
    action_step_size=0.05,
    closing_prices=np.random.rand(2000, 3),  # ✅ Use actual data
    reward_method="portfolio_value",
    prediction_model=my_trained_model  # ✅ ML model for predictions
)

# Run PPO directly
ppo.main(args)


------------ REACHED ENVIROMENT SETUP ------------
🔍 Observation space: Box([  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf], [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1. inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf], (40,), float64)
🔍 Observation space shape: (40,)
------------ FINISHED ENVIRONMENT SETUP ------------
global_step=500, episodic_return=[9.835805e+16]
global_step=500, episodic_return=[8.6457273e+15]
global_step=500, episodic_return=[2.4528235e+19]
global_step=500, episodic_return=[7.800497e+16]
global_step=500, episodic_return=[2.9835225e+14]
SPS: 2353
global_step=1000, episodic_return=[1.6523771e+22]
global_step=1000, episodic_return=[8.2237835e+16]
global_step=1000, episodic_return=[4.51153e+16]
global_step=1000, episodic_return=[2.8666428e+20]
g